## Preprocessing 

In [1]:
# Import packages
import os
import glob
import collections
import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm
import gensim
import pyLDAvis.gensim
from gensim.models import Phrases
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from nltk.corpus import stopwords
from stop_words import get_stop_words
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""


In [ ]:
# Initialise spaCy
nlp = spacy.load('en')

In [ ]:
# Read in CSVs of abstracts
path = "../data/elsevier/Elsevier_EnergyPolicy.csv"
df = pd.read_csv(path)

In [ ]:
elsevier = glob.glob("data/elsevier/*.csv")
wiley = glob.glob("data/wiley/*.csv")

In [7]:
allElsevier = glob.glob("~Document/digital_literacies/projects/vladimir/TopicModelling/data/elsevier/*.csv") 
allWiley = glob.glob("~Document/digital_literacies/projects/vladimir/TopicModelling/data/wiley/*.csv") 
list_ = []

for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)

data = pd.concat(list_, axis = 0, ignore_index = True)

SyntaxError: unexpected character after line continuation character (<ipython-input-7-f15e0954c9cd>, line 1)

In [16]:
allElsevier = glob.glob("/Users/au564346/Documents/research/digital_literacies/projects/vladimir/TopicModelling/data/elsevier/*.csv") 
allWiley = glob.glob("/Users/au564346/Documents/research/digital_literacies/projects/vladimir/TopicModelling/data/wiley/*.csv") 


In [18]:
allElsevier

['/Users/au564346/Documents/research/digital_literacies/projects/vladimir/TopicModelling/data/elsevier/Elsevier.Energy_Procedia.csv',
 '/Users/au564346/Documents/research/digital_literacies/projects/vladimir/TopicModelling/data/elsevier/Elsevier.Resources_Policy.csv',
 '/Users/au564346/Documents/research/digital_literacies/projects/vladimir/TopicModelling/data/elsevier/Elsevier_EnergyForSustainableDevelopment.csv',
 '/Users/au564346/Documents/research/digital_literacies/projects/vladimir/TopicModelling/data/elsevier/Elsevier_EnergyInternationalJournal.csv',
 '/Users/au564346/Documents/research/digital_literacies/projects/vladimir/TopicModelling/data/elsevier/Elsevier_EnergyPolicy.csv',
 '/Users/au564346/Documents/research/digital_literacies/projects/vladimir/TopicModelling/data/elsevier/Elsevier_EnergyResearch&SocialScience.csv',
 '/Users/au564346/Documents/research/digital_literacies/projects/vladimir/TopicModelling/data/elsevier/Elsevier_Resource&EnergyEconomics.csv']

In [ ]:
stop_1 = get_stop_words("english")
stop_2 = stopwords.words('english')
stopwords = list(set(stop_1 + stop_2))
stopwords.extend(["-PRON-"])

In [ ]:
len(df)

In [ ]:
# Document preprocesing
import time
# Main pipeline
processed_docs = [] 

n_threads = 3
batch_size = 100
for doc in tqdm(nlp.pipe(df["abstract"][0:100], n_threads=n_threads, batch_size=batch_size), total=batch_size):
    
    # Named entities.
    ents = doc.ents

    # Keep only words (no numbers, no punctuation).
    # Lemmatize tokens, remove punctuation and remove spaCy stopwords.
    doc = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]

    # Remove common words from a stopword list.
    doc = [token for token in doc if token not in stopwords]

    # Add named entities, but only if they are a compound of more than word. E.g. - United_States, European_Union
    doc.extend([str(entity) for entity in ents if len(entity) > 1])
    
    processed_docs.append(doc)

# Rename and delete
docs = processed_docs
del processed_docs

In [ ]:
# Compute bigrams & trigrams
bigram = Phrases(docs, min_count=5)
trigram = Phrases(bigram[docs])
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [ ]:
# Create a dictionary representation of the documents, and filter out frequent and rare words.
dictionary = Dictionary(docs)

In [ ]:
# Remove rare and common tokens.
max_freq = 0.8
min_wordcount = 10
dictionary.filter_extremes(no_below=min_wordcount, no_above=max_freq)
# This sort of "initializes" dictionary.id2token.
_ = dictionary[0]

In [ ]:
# Vectorize data.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [ ]:
# Inspect elements
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

## Experimenting with topic coherence

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Multiple random initializations
model_list = []
for i in range(5):
    model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, 
            update_every=update_every, chunksize=chunksize, passes=passes, eval_every=eval_every)
    top_topics = model.top_topics(corpus)
    tc = sum([t[1] for t in top_topics])
    model_list.append((model, tc))

In [ ]:
# Choose most coherent model and display coherence
model, tc = max(model_list, key=lambda x: x[1])
print('Topic coherence: %.3e' %tc)

## Inspecting the topics

In [ ]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

In [ ]:
get_lda_topics(model, num_topics)

In [ ]:
# Define function to present topics in a neater format
def explore_topic(lda_model, topic_number, topn, output=True):
    """
    accept a ldamodel, atopic number and topn vocabs of interest
    prints a formatted list of the topn terms
    """
    terms = []
    for term, frequency in lda_model.show_topic(topic_number, topn=topn):
        terms += [term]
        if output:
            print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))
    
    return terms

In [ ]:
# Display neat topics
topic_summaries = []
print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')

topic_summaries = []
for i in range(num_topics):
    print('Topic '+str(i)+' |---------------------\n')
    tmp = explore_topic(model,topic_number=i, topn=10, output=True )
#     print tmp[:5]
    topic_summaries += [tmp[:5]]
    print(topic_summaries)

In [ ]:
# Apply user defined labels to topics
top_labels = {0: 'Fossil fuel', 1:'Nuclear energy', 2:'Efficient consumption', 3:'Public policy', 4:'Cimate change', 5:'United States',
             6:'Energy Security', 7:'Energy transition', 8:'Academic Research', 9:'International Development'}

## Visualisation

In [ ]:
# Initialize pyLDAvis
pyLDAvis.enable_notebook()

In [ ]:
# Inspect model in pyLDAvis
pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=False)

In [ ]:
# Inspect model in pyLDAvis
pyLDAvis.save_html(pyLDAvis.gensim.prepare(model, corpus, dictionary), 'lda.html')

---------------


##### Everything below here is experimental. Beware dragons!

---------------

# Plotting topics use T-SNE 

In [ ]:
import re
import nltk
from nltk.stem import WordNetLemmatizer

In [ ]:
# Define useful functions

def article_to_wordlist(article, remove_stopwords=True ):
    '''
    Word tokenizer for articles using NLTK
    '''
    
    lemmatizer = WordNetLemmatizer()
    # 1. Remove non-letters
    article_text = re.sub("[^a-zA-Z]"," ", article)
    # 2. Convert words to lower case and split them
    words = article_text.lower().split()
    # 3. Remove stop words
    words = [w for w in words if not w in stopwords]
    # 4. Remove short words
    words = [t for t in words if len(t) > 2]
    # 5. lemmatizing
    words = [nltk.stem.WordNetLemmatizer().lemmatize(t) for t in words]

    return(words)

# 
def get_doc_topic_dist(model, corpus, kwords=False):
    '''
    Creates an array containing all non-zero doc-topic distributions
    '''
    
    top_dist =[]
    keys = []

    for d in corpus:
        tmp = {i:0 for i in range(num_topics)}
        tmp.update(dict(model[d]))
        vals = list(collections.OrderedDict(tmp).values())
        top_dist += [np.array(vals)]
        if kwords:
            keys += [np.array(vals).argmax()]

    return np.array(top_dist), keys

In [ ]:
# Create tfvectorizer instance
from sklearn.feature_extraction.text import TfidfVectorizer

tvectorizer = TfidfVectorizer(input='content', analyzer = 'word', lowercase=True, stop_words='english',\
                                  tokenizer=article_to_wordlist, ngram_range=(1, 3), min_df=20, max_df=0.80,\
                                  norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=True)

In [ ]:
# Create Document-Term Matrix using Tf-idf scores
top_dist, lda_keys= get_doc_topic_dist(model, corpus, True)
dtm = tvectorizer.fit_transform(df['abstract']).toarray()
features = tvectorizer.get_feature_names()

In [ ]:
top_ws = []
for n in range(len(dtm)):
    inds = np.int0(np.argsort(dtm[n])[::-1][:4])
    tmp = [features[i] for i in inds]
    
    top_ws += [' '.join(tmp)]
    
df['Text_Rep'] = pd.DataFrame(top_ws)
df['clusters'] = pd.DataFrame(lda_keys)
df['clusters'].fillna(10, inplace=True)

cluster_colors = {0: 'blue', 1: 'green', 2: 'yellow', 3: 'red', 4: 'skyblue', 5:'salmon', 6:'orange', 7:'maroon', 8:'crimson', 9:'black', 10:'gray'}

df['colors'] = df['clusters'].apply(lambda l: cluster_colors[l])

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, perplexity=30)
X_tsne = tsne.fit_transform(top_dist)

In [ ]:
df['X_tsne'] =X_tsne[:, 0]
df['Y_tsne'] =X_tsne[:, 1]

In [ ]:
from bokeh.plotting import figure, show, output_notebook, save
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource
output_notebook()

In [ ]:
source = ColumnDataSource(dict(
    x = df['X_tsne'],
    y = df['Y_tsne'],
    color = df['colors'],
    label = df['clusters'].apply(lambda l: top_labels[l]),
#     msize = df['marker_size'],
    topic_key = df['clusters'],
   #title = df['title'],
    content = df['Text_Rep']
))

In [ ]:
title = 'T-SNE visualization of topics'

plot_lda = figure(plot_width=1000, plot_height=600,
                     title=title, tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
                     x_axis_type=None, y_axis_type=None, min_border=1)

plot_lda.scatter(x='x', y='y', legend='label', source=source, color='color', alpha=0.8, size=10)#'msize', )

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "Title: @title, KeyWords: @content - Topic: @topic_key "}
plot_lda.legend.location = "top_left"

show(plot_lda)

In [ ]:
#save the plot
save(plot_lda, '{}.html'.format(title))